# Lambda calculus
introduction from [David Baezly - Lambda Calculus: PyCon 2019 Tutorial (Screencast)](https://www.youtube.com/watch?v=5C6sv7-eTKg)

only single argument functions exists

[Jupyternotebook hotkeys](https://bbyond.medium.com/vscode-jupyter-notebook-keyboard-shortcuts-31fab95fa301)

## The switch

In [6]:
def Left(a):
    def f(b):
        return a
    return f

def Right(a):
    def f(b):
        return b
    return f

In [7]:
Left('A')('B')

'A'

In [8]:
Right('A')('B')

'B'

## True and False

In [13]:
def TRUE(x):
    return lambda y: x

def FALSE(x):
    return lambda y: y

In [14]:
TRUE('a')('b')

'a'

In [15]:
FALSE('a')('b')

'b'

In [29]:
# assert TRUE(TRUE) is TRUE
# assert TRUE(FALSE) is FALSE

# assert FALSE(TRUE) is FALSE
# assert FALSE(FALSE) is TRUE


In [35]:
# TRUE(TRUE)

In [36]:
# FALSE(FALSE)

## Boolen operators


In [17]:
def NOT(x):
    return x(FALSE)(TRUE)

assert NOT(TRUE) is FALSE
assert NOT(FALSE) is TRUE


In [18]:
NOT(TRUE)

<function __main__.FALSE(x)>

In [19]:
NOT(FALSE)

<function __main__.TRUE(x)>

In [23]:
def AND(x):
    return lambda y: x(y)(x)

assert AND(TRUE)(TRUE) is TRUE
assert AND(TRUE)(FALSE) is FALSE
assert AND(FALSE)(TRUE) is FALSE
assert AND(FALSE)(FALSE) is FALSE

def OR(x):
    # return lambda y: NOT(x(NOT(x))(NOT(y)))
    return lambda y: x(x)(y)

assert OR(TRUE)(TRUE) is TRUE
assert OR(TRUE)(FALSE) is TRUE
assert OR(FALSE)(TRUE) is TRUE
assert OR(FALSE)(FALSE) is FALSE

## The numbers

In [56]:
ONE  = lambda f: lambda x: f(x) # one usage of x
TWO = lambda f: lambda x: f(f(x)) # two usage of x
THREE = lambda f: lambda x: f(f(f(x)))
FOUR = lambda f: lambda x: f(f(f(f(x))))

For understanding let's define `incr` and play with it. This is needed to be able to explain it in python.

In [51]:
def incr(x):
    return x+1 # this is illegal (+) in lambda calculus

THREE(incr)(0)

3

let's define a tuple: 
THREE is works like counting repeating the tuple 3 times

In [48]:
def p(t):
    return (t[0]+1, t[1])

THREE(p)((2,1))

(5, 1)

How we add more numbers? 

In [54]:
a = FOUR(THREE)
a(incr)(0)

81

This is $3^4$

In [55]:
a(incr)(1)

82

implementation of Zero? WE will see it is similar than `FALSE`

In [58]:
ZEROS = lambda f: lambda x: x # no usage of x

ZEROS(incr)(0)

0

In [81]:
TRUE = lambda f: lambda x: f(x)
FALSE = lambda f: lambda x: x 

assert FALSE(incr)(0) == 0
assert TRUE(incr)(0) == 1


What is the most minimal thing to do almost anything in math? Like count. (Peano)

* $0$ is a number
* If $n$ is a number, then the successor of $n$ is a number
* $0$ is not the successor of any number
* $succ(n_1) = succ(n_2)$ implies $n_1 = n_2$
* If $S$ is a set that contains $0$ and the successor of every number in $S$, then $S$ is the set of all numbers

Challenge: implement _successor_

In [59]:
SUCC = lambda n: lambda f: lambda x: f(n(f)(x))

In [60]:
SUCC(FOUR)

<function __main__.<lambda>.<locals>.<lambda>(f)>

In [61]:
_(incr)(0)

5

In [64]:
SUCC(SUCC(FOUR))(incr)(0)

6

In [65]:
ADD = lambda x: lambda y: y(SUCC)(x)

In [67]:
ADD(FOUR)(THREE)(incr)(0)

7

In [68]:
MUL = lambda x: lambda y : lambda f: y(x(f))

In [70]:
MUL(FOUR)(THREE)
_(incr)(0)

12

In [71]:
def show(n): # debugging only not lambda calculus
    print(n(incr)(0))

## The digression

recall `TWO = lambda f: lambda x: f(f(x))` and see a nested dictionary example

In [82]:
data = {
    'a': {
        'b': {
            'c': 42
        }
    }
}

def getc(d):
    return d['a']['b']['c']

getc(data)

42

to make this tolerating malformulated dictionaries, we can have the following  

In [84]:
def getc(d):
    d = d.get('a')
    if d is not None:
        d = d.get('b')
    if d is not None:
        d = d.get('c')
    return d

getc({})

In [85]:
def perhaps(d, func):
    if d is not None:
        return func(d)
    else:
        return None

perhaps(data, lambda d:d.get('a'))


{'b': {'c': 42}}

and then `perhaps` can be used chaining to get `42`, but this is not so useable in reality

In [87]:
class Perhaps: # classes is not part of lambda calculus
    def __init__(self, value):
        self.value = value

    def __rshift__(self, other):
        if self.value is not None:
            return Perhaps(other(self.value))
        else:
            return self

# Perhaps this is an example of a "Monad"

Perhaps(data) >> (lambda d: d.get('a')) >> (lambda d: d.get('b')) >> (lambda d: d.get('c'))
_.value

42

1:19 Break